In [14]:
import requests

In [15]:
REGION = 'us'
ODDS_FORMAT = 'decimal'

with open("api_key.txt") as f:
    API_KEY = f.read().strip()


In [16]:
# Gets the list of sports 
def get_sports(api_key=API_KEY):
    url = f'https://api.the-odds-api.com/v4/sports?apiKey={api_key}'
    r = requests.get(url).json()
    return list(map(lambda sport: sport['key'], r))
    

In [17]:
# Gets the odds for a sport
def get_odds(sport_id, region=REGION, api_key=API_KEY, odds_format=ODDS_FORMAT):
    url = f"https://api.the-odds-api.com/v4/sports/{sport_id}/odds?regions={region}&oddsFormat={odds_format}&apiKey={api_key}"
    r = requests.get(url).json()
    return r

In [18]:
# Processes the data in tidy format
def process(sports):

    # Formats datapoints as dictionaries and stores them in all_events
    all_events = []
    event_ids = []
    for sport in sports:
        for event in sports[sport]:
            event_ids.append(event['id'])
            for bookmaker in event['bookmakers']:
                for market in bookmaker['markets']:
                    for outcome in market['outcomes']:
                        new_event = {}
                        new_event['id'] = event['id']
                        new_event['sport_title'] = event['sport_title']
                        new_event['date'] = event['commence_time']
                        new_event['bookmaker'] = bookmaker['title']
                        new_event['market'] = market['key']
                        new_event['outcome'] = outcome['name']
                        new_event['price'] = outcome['price']
                        all_events.append(new_event)
    
    return all_events , event_ids

In [19]:
def get_best_odds(event_id, events):
    filter_events = list(filter(lambda event: event['id'] == event_id, events))
    outcomes = set(list(map(lambda event: event['outcome'],filter_events)))
    best_odds = []
    total = 0
    for outcome in outcomes:
        filter_outcomes = list(filter(lambda event: event['outcome'] == outcome, filter_events))
        best = max(filter_outcomes, key = lambda x: x['price'])
        best_odds.append(best)
        total += 1/best['price']
    if total != 0:
        return {"sport_title" : best_odds[0]['sport_title'],
                "date": best_odds[0]['date'],
                "outcomes": list(outcomes),
                "bets": [{"bookmaker": odds['bookmaker'], "outcome": odds['outcome'], "price":odds['price']} 
                         for odds in best_odds],
                "return": round(100-total*100, 3)}

In [20]:
def print_odds(odds):
    print(f"Sport: {odds['sport_title']}\n")
    outcomes = ", ".join(odds['outcomes'])
    print(f"Outcomes: {outcomes}, Date: {odds['date']}\n")
    scaling_factor = 1 / sum(list(map(lambda x: 1/ x['price'], odds['bets'])))
    for bets in odds['bets']:
        bet_percentage = round(1/bets['price'] * scaling_factor,3)
        print(f"{bets['bookmaker']}, {bets['outcome']}, Bet Percentage: {bet_percentage}, Payout: {bets['price']}x\n")
    print(f"Arbitrage Return: {odds['return']/2}%\n\n")

In [21]:
# Gets the odds for all events in all sports
sports_odds = {sport:get_odds(sport) for sport in get_sports()}

# Processes event data into a list of dictionaries
all_events, event_ids = process(sports_odds)

# Finds and sorts the best odds for each event
best_odds = []
for event_id in event_ids:
    best_odds_event = get_best_odds(event_id, all_events)
    if best_odds_event is not None:
        best_odds.append(best_odds_event) 

best_odds.sort(key = lambda x:x['return'],reverse=True)

# Displays events with sure bet opportunites
best_odds = list(filter(lambda odds: odds['return'] > 0, best_odds))
for odds in best_odds:
    print_odds(odds)

Sport: Primeira Liga - Portugal

Outcomes: FC Porto, Draw, Braga, Date: 2025-11-02T20:31:00Z

DraftKings, FC Porto, Bet Percentage: 0.528, Payout: 3.25x

Bovada, Draw, Bet Percentage: 0.358, Payout: 4.8x

Bovada, Braga, Bet Percentage: 0.114, Payout: 15.0x

Arbitrage Return: 20.8655%


Sport: Brazil Série B

Outcomes: Chapecoense, Draw, Remo, Date: 2025-11-02T21:31:00Z

BetMGM, Chapecoense, Bet Percentage: 0.151, Payout: 8.75x

BetMGM, Draw, Bet Percentage: 0.314, Payout: 4.2x

FanDuel, Remo, Bet Percentage: 0.536, Payout: 2.46x

Arbitrage Return: 12.0555%


Sport: Brazil Série A

Outcomes: Draw, Atletico Mineiro, Internacional, Date: 2025-11-02T21:29:00Z

DraftKings, Draw, Bet Percentage: 0.334, Payout: 3.45x

DraftKings, Atletico Mineiro, Bet Percentage: 0.096, Payout: 12.0x

Bovada, Internacional, Bet Percentage: 0.57, Payout: 2.02x

Arbitrage Return: 6.588%


Sport: Brazil Série A

Outcomes: Palmeiras, Draw, Juventude, Date: 2025-11-02T21:33:00Z

Bovada, Palmeiras, Bet Percentage: 